In [0]:
import requests
import json
import os
import re
import yaml

# --- Configuration ---
# Configuration
CATALOG = "dbx_migration_poc"
SCHEMA = "dbx_migration_ts"
TML_VOLUME = "lv_dashfiles_ak"
LVDASH_VOLUME = "lvdash_files_ak_out"
TS_URL = "https://team2.thoughtspot.cloud"
TS_TOKEN = "88ea004e-b296-4373-bf79-1213bf3e0a8a" # Recommended: Fetch this from Databricks Secrets/Key Vault
LIVEBOARD_GUID = "fc2fbe37-6b73-4831-a11d-28602afc84fc"
OUTPUT_VOLUME_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/{TML_VOLUME}/tml_exports/"

# 1. Helper: Parse name from TML content (The Source of Truth)
def get_name_from_tml(tml_string):
    try:
        # Parse the YAML content
        data = yaml.safe_load(tml_string)
        
        # TML usually has one root key like 'liveboard', 'worksheet', 'view'
        # e.g., { 'liveboard': { 'name': 'My Dashboard', ... } }
        if isinstance(data, dict):
            # Get the first key (the object type)
            root_key = next(iter(data)) 
            # Dig into that object to find 'name'
            if isinstance(data[root_key], dict):
                return data[root_key].get('name', 'Unnamed')
                
    except Exception:
        pass
    return "Unknown_Object"

# 2. Helper: Sanitize for filename
def sanitize_filename(name):
    # Remove special chars and spaces
    return re.sub(r'[\\/*?:"<>| ]', '_', name)

def get_thoughtspot_token(base_url, username, password):
    """Generates a fresh Bearer token using V2 API"""
    auth_url = f"{base_url}/api/rest/2.0/auth/token/full"
    
    # Payload for getting a token
    payload = {
        "username": username,
        "password": password,
        "validity_time_in_sec": 300  # 5 minutes is usually enough for the script
    }
    
    headers = {"Content-Type": "application/json", "Accept": "application/json"}
    
    try:
        response = requests.post(auth_url, headers=headers, json=payload)
        response.raise_for_status()
        # The response structure depends on version, usually it's simply 'token' 
        # or inside a detailed object.
        return response.json().get("token") 
    except Exception as e:
        print(f"Auth Failed: {e}")
        print(response.text)
        raise

# Get fresh token
TS_TOKEN = get_thoughtspot_token(TS_URL, ts_user, ts_pass)
print("Generated fresh token.")

# --- API Request ---
url = f"{TS_URL}/api/rest/2.0/metadata/tml/export"

headers = {
    "Authorization": f"Bearer {TS_TOKEN}",
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# --- Corrected Payload ---
payload = {
    "metadata": [
        {
            "identifier": LIVEBOARD_GUID,
            "type": "LIVEBOARD" 
        }
    ],
    # Move these to the root level
    "export_associated": True,  
    "format_type": "YAML"       
}

try:
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    
    # The V2 API returns a list of objects
    exported_items = response.json() 

    print(f"Processing {len(exported_items)} objects...")

    for item in exported_items:
        # 2. Extract the true name from the metadata wrapper
        # The API guarantees a 'name' field in the response object
        raw_name = item.get('name', 'Unknown_Object')
        obj_type = item.get('type', 'Unknown_Type')
        
        # 3. Sanitize the name
        safe_name = sanitize_filename(raw_name)
        
        # Optional: Prefix with type to avoid collisions (e.g., 'WORKSHEET_Sales_Data.tml')
        # or just use the name if you prefer: file_name = f"{safe_name}.tml"
        file_name = f"{obj_type}_{safe_name}.tml"
        
        # 4. Extract content
        # V2 API typically puts the TML string in 'edoc' or 'content'
        tml_content = item.get('edoc', item.get('content'))
        
        if tml_content:
            full_path = os.path.join(OUTPUT_VOLUME_PATH, file_name)
            
            with open(full_path, "w", encoding='utf-8') as f:
                f.write(tml_content)
            
            print(f"Successfully saved: {file_name}")
        else:
            print(f"Warning: No TML content found for {raw_name}")

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")